# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [6]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import yfinance as yf

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
stock = yf.Ticker('AAPL')
data = stock.info
data

In [12]:
print(f"Current or Latest Price of the stock: {data['currentPrice']}")
print(f"1 year Change: {data['52WeekChange']}")

Current or Latest Price of the stock: 217.53
1 year Change: 0.20494819


In [13]:
# Initialize an empty list to store the results
data = []

# Iterate through each stock symbol in the CSV file
for symbol in stocks['Ticker']:
    try:
        # Fetch the stock data using yfinance
        stock = yf.Ticker(symbol)
        stock_info = stock.info

        # Get the stock price and market cap
        stock_price = stock_info.get('currentPrice', None)
        one_year_price_return = stock_info.get('52WeekChange', None)

        # Append the data to the list
        data.append([symbol, stock_price, one_year_price_return, 0])

    except Exception as e:
        print(f"Failed to get data for {symbol}: {e}")
        data.append([symbol, None, None, 0])

# Create a DataFrame from the collected data
data = pd.DataFrame(data, columns= ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy'])

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=RNEJFNig9cq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.

In [50]:
final_dataframe = data.copy(deep=True)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [51]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,109.02,1.394122,0
1,NRG,79.87,1.161570,0
2,GPS,22.15,1.153042,0
3,HWM,92.83,0.891304,0
4,ANET,338.33,0.885302,0
5,SLG,60.76,0.885200,0
6,GE,166.87,0.812323,0
7,IRM,108.67,0.801030,0
8,PGR,222.99,0.760880,0
9,AVGO,148.62,0.735862,0


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [52]:
portfolio_size = 100000

In [53]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,109.02,1.394122,17
1,NRG,79.87,1.161570,24
2,GPS,22.15,1.153042,88
3,HWM,92.83,0.891304,21
4,ANET,338.33,0.885302,5
5,SLG,60.76,0.885200,32
6,GE,166.87,0.812323,11
7,IRM,108.67,0.801030,18
8,PGR,222.99,0.760880,8
9,AVGO,148.62,0.735862,13


In [84]:
ticker = 'WDC'
stock_data = yf.Ticker(ticker)

# Get historical market data (1 year)
hist = stock_data.history(period="1y")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-14 00:00:00-04:00,42.110001,42.730000,42.090000,42.599998,2016500,0.0,0.0
2023-08-15 00:00:00-04:00,42.240002,42.270000,41.029999,41.049999,2987600,0.0,0.0
2023-08-16 00:00:00-04:00,40.759998,40.869999,40.099998,40.419998,2366700,0.0,0.0
2023-08-17 00:00:00-04:00,40.639999,41.279999,40.130001,40.290001,2253800,0.0,0.0
2023-08-18 00:00:00-04:00,39.509998,39.950001,39.230000,39.740002,2405800,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-06 00:00:00-04:00,56.619999,58.799999,56.619999,57.380001,9543700,0.0,0.0
2024-08-07 00:00:00-04:00,58.650002,59.529999,55.730000,56.250000,8197500,0.0,0.0
2024-08-08 00:00:00-04:00,57.590000,59.439999,56.459999,58.980000,6067000,0.0,0.0


In [85]:
print(hist['Open'].iloc[0])
print(hist['Close'].iloc[0])

print(hist['Open'].iloc[-1])
print(hist['Close'].iloc[-1])

42.11000061035156
42.599998474121094
59.68000030517578
60.369998931884766


In [86]:
(hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]

0.4171361759216658

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [46]:
# Store returns for percentile calculation
all_returns = {
    'One-Year': [],
    'Six-Month': [],
    'Three-Month': [],
    'One-Month': []
}

# Fetch data and calculate metrics for each ticker
for i in range(len(final_dataframe)):
    ticker = final_dataframe['Ticker'][i]
    stock_data = yf.Ticker(ticker)
    
    # Get historical market data
    hist = stock_data.history(period="1y")
    
    # Initialize returns
    one_year_return = None
    six_month_return = None
    three_month_return = None
    one_month_return = None
    
    # Check if enough data is available
    if len(hist) < 30:  # e.g., need at least 30 days of data for calculations
        continue

    # Calculate returns
    if len(hist) >= 252:  # Approx. 252 trading days in a year
        one_year_return = (hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]
        final_dataframe.loc[i, 'One-Year Price Return'] = one_year_return
        all_returns['One-Year'].append(one_year_return)

    if len(hist) >= 126:  # Approx. 126 trading days in six months
        six_month_return = (hist['Close'][-1] - hist['Close'][-126]) / hist['Close'][-126]
        final_dataframe.loc[i, 'Six-Month Price Return'] = six_month_return
        all_returns['Six-Month'].append(six_month_return)

    if len(hist) >= 63:  # Approx. 63 trading days in three months
        three_month_return = (hist['Close'][-1] - hist['Close'][-63]) / hist['Close'][-63]
        final_dataframe.loc[i, 'Three-Month Price Return'] = three_month_return
        all_returns['Three-Month'].append(three_month_return)

    if len(hist) >= 21:  # Approx. 21 trading days in one month
        one_month_return = (hist['Close'][-1] - hist['Close'][-21]) / hist['Close'][-21]
        final_dataframe.loc[i, 'One-Month Price Return'] = one_month_return
        all_returns['One-Month'].append(one_month_return)

    # Example HQM Score calculation (replace with actual logic)
    # For now, set to 0 as placeholder
    final_dataframe.loc[i, 'HQM Score'] = 0.0

# Calculate percentiles
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

# for time_period in time_periods:
#     percentile_column = f'{time_period} Return Percentile'
#     return_column = f'{time_period} Price Return'
#     final_dataframe[percentile_column] = final_dataframe.apply(
#         lambda row: stats.percentileofscore(all_returns[time_period], row[return_column]) / 100,
#         axis=1
    # )

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(final_dataframe[f'{time_period} Return Percentile'])

/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/712073052.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  six_month_return = (hist['Close'][-1] - hist['Close'][-126]) / hist['Close'][-126]
/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/712073052.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  three_month_return = (hist['Close'][-1] - hist['Close'][-63]) / hist['Close'][-63]
/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/712073052.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alway

In [38]:
(hist['Close'][-1] - hist['Close'][0]) / (hist['Close'][0])

/var/folders/7w/mm9ktp7n75dgn8rjth6_zv_m0000gn/T/ipykernel_64412/2058894446.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (hist['Close'][-1] - hist['Close'][0]) / (hist['Close'][0])


0.38683534853491613

In [48]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,HQM Score
0,NVDA,109.02,1.394122,17,0.509164,0.206086,-0.151199,NaN,0.92,0.92,0.06,0.0
1,NRG,79.87,1.161570,24,0.560112,-0.026123,0.065206,NaN,0.94,0.10,0.84,0.0
2,GPS,22.15,1.153042,88,0.091296,0.001514,-0.035279,NaN,0.16,0.26,0.32,0.0
3,HWM,92.83,0.891304,21,0.580281,0.158200,0.177577,NaN,0.96,0.78,0.96,0.0
4,ANET,338.33,0.885302,5,0.204536,0.091845,-0.061290,NaN,0.54,0.58,0.22,0.0
5,SLG,60.76,0.885200,32,0.372844,0.161521,-0.039408,NaN,0.84,0.82,0.26,0.0
6,GE,166.87,0.812323,11,0.496858,0.047978,0.045813,NaN,0.90,0.40,0.72,0.0
7,IRM,108.67,0.801030,18,0.603576,0.396587,0.110804,NaN,1.00,1.00,0.88,0.0
8,PGR,222.99,0.760880,8,0.223963,0.050307,0.027130,NaN,0.60,0.44,0.62,0.0
9,AVGO,148.62,0.735862,13,0.183612,0.114697,-0.133007,NaN,0.40,0.68,0.08,0.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.885149
1      0.0237624
2       0.578218
3       0.992079
4        0.89703
         ...    
500     0.211881
501     0.457426
502     0.843564
503     0.255446
504     0.772277
Name: One-Year Return Percentile, Length: 505, dtype: object
0       0.837624
1      0.0158416
2       0.839604
3       0.968317
4       0.629703
         ...    
500     0.405941
501      0.39604
502     0.906931
503     0.227723
504     0.776238
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.473267
1      0.908911
2      0.643564
3      0.887129
4       0.19802
         ...   
500    0.374257
501    0.544554
502    0.611881
503     0.70297
504    0.665347
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.530693
1       0.827723
2       0.742574
3       0.879208
4      0.0693069
         ...    
500     0.370297
501     0.762376
502     0.641584
503     0.312871
504     0.792079
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,98.19,N/A,0.444090,0.885149,0.147456,0.837624,0.221461,0.473267,0.093820,0.530693,N/A
1,AAL,13.89,N/A,-0.526494,0.0237624,-0.564540,0.0158416,0.509431,0.908911,0.172430,0.827723,N/A
2,AAP,161.13,N/A,0.088066,0.578218,0.148378,0.839604,0.295700,0.643564,0.144608,0.742574,N/A
3,AAPL,467.65,N/A,1.171724,0.992079,0.401695,0.968317,0.474900,0.887129,0.189840,0.879208,N/A
4,ABBV,97.29,N/A,0.478770,0.89703,0.001711,0.629703,0.077880,0.19802,-0.024533,0.0693069,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,93.51,N/A,-0.214524,0.211881,-0.116757,0.405941,0.161550,0.374257,0.068180,0.370297,N/A
501,ZBH,140.16,N/A,0.003007,0.457426,-0.127491,0.39604,0.250906,0.544554,0.151414,0.762376,N/A
502,ZBRA,285.97,N/A,0.373952,0.843564,0.223856,0.906931,0.283668,0.611881,0.115379,0.641584,N/A
503,ZION,34.95,N/A,-0.161814,0.255446,-0.255398,0.227723,0.328190,0.70297,0.055265,0.312871,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,98.19,199,0.444090,0.885149,0.147456,0.837624,0.221461,0.473267,0.093820,0.530693,0.681683
1,AAL,13.89,1411,-0.526494,0.0237624,-0.564540,0.0158416,0.509431,0.908911,0.172430,0.827723,0.444059
2,AAP,161.13,121,0.088066,0.578218,0.148378,0.839604,0.295700,0.643564,0.144608,0.742574,0.70099
3,AAPL,467.65,41,1.171724,0.992079,0.401695,0.968317,0.474900,0.887129,0.189840,0.879208,0.931683
4,ABBV,97.29,201,0.478770,0.89703,0.001711,0.629703,0.077880,0.19802,-0.024533,0.0693069,0.448515
5,ABC,104.97,186,0.163705,0.651485,0.100112,0.774257,0.241867,0.522772,0.066990,0.364356,0.578218
6,ABMD,305.78,64,0.546138,0.942574,0.874783,0.99802,0.645795,0.954455,0.145153,0.746535,0.910396
7,ABT,102.94,190,0.161073,0.647525,0.139130,0.823762,0.093917,0.233663,0.081815,0.457426,0.540594
8,ACN,237.88,82,0.197484,0.69505,0.085519,0.762376,0.279570,0.605941,0.067045,0.366337,0.607426
9,ADBE,453.91,43,0.532839,0.934653,0.196705,0.879208,0.252382,0.550495,0.006829,0.150495,0.628713


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()